In [1]:
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from sklearn import linear_model
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics.pairwise import sigmoid_kernel, rbf_kernel, polynomial_kernel
from sklearn import neighbors
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict


## 1. Load Data

In [2]:
# Load the features
X = np.loadtxt(open("AD_MRI_DATA/3-MCI_AD/3.3-MRI_Volume.csv","rb"),delimiter=",",skiprows=0)
X = np.array(X, dtype=float)
print("X:" + str(X.shape))

# Load the labels
with open('AD_MRI_DATA/3-MCI_AD/3-Phenotype.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    c1 = [row[1]for row in reader]
    
del c1[0]
y = np.array(c1, dtype=int)
print("y:" + str(y.shape))


X:(146, 109)
y:(146,)


## 2. Lasso with Classifiers

### 2.1 Lasso

In [3]:
model = linear_model.LassoCV()
#alphas=[1,0.1,0.01,0.005,0.001,0.0001]
model.fit(X, y)

print("alpha: " + str(model.alpha_))
#print(lasso.coef_)
3
selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected = selector.transform(X)
print("X_selected: " + str(X_selected.shape))

alpha: 17.240071307937708
X_selected: (146, 16)


### 2.2 Classifiers

In [4]:
for i in range(5):
    # SVM
    if i == 0:
        clf = SVC(C=1, random_state=42, probability=True)
        print("SVM")
        
        
    # KNN
    elif i == 1:
        clf = neighbors.KNeighborsClassifier(n_neighbors=2)
        print("KNN")
        
    # Gaussian Naive Bayes
    elif i == 2:
        clf = GaussianNB()
        print("Gaussian Naive Bayes")
        
    # Decision Tree
    elif i == 3:
        clf = tree.DecisionTreeClassifier()
        print("Decision Tree")
        
    # Random Forest
    else:
        clf = RandomForestClassifier()
        print("Random Forest")
    
    # Model Evaluation
    y_pred = cross_val_predict(clf, X_selected, y, cv=5)
    auc = cross_val_score(clf, X_selected, y, cv=5, scoring="roc_auc")
    print(confusion_matrix(y, y_pred,labels=[1,0]))
    print(classification_report(y, y_pred))
    auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
    print("auc: "+str(auc_avg)+"\n")


SVM
[[  8  38]
 [  0 100]]
              precision    recall  f1-score   support

           0       0.72      1.00      0.84       100
           1       1.00      0.17      0.30        46

    accuracy                           0.74       146
   macro avg       0.86      0.59      0.57       146
weighted avg       0.81      0.74      0.67       146

auc: 0.9661111111111111

KNN
[[25 21]
 [ 2 98]]
              precision    recall  f1-score   support

           0       0.82      0.98      0.89       100
           1       0.93      0.54      0.68        46

    accuracy                           0.84       146
   macro avg       0.87      0.76      0.79       146
weighted avg       0.86      0.84      0.83       146

auc: 0.855111111111111

Gaussian Naive Bayes
[[35 11]
 [ 7 93]]
              precision    recall  f1-score   support

           0       0.89      0.93      0.91       100
           1       0.83      0.76      0.80        46

    accuracy                           0.88

### 2.3 Multi-core SVM

In [5]:
sig_mat = sigmoid_kernel(X_selected)
rbf_mat = rbf_kernel(X_selected)
poly_mat = polynomial_kernel(X_selected)

best_score = 0
best_w1 = 0
best_w2 = 0
best_w3 = 0

for i in range(11):
    for j in range(11):
        w1 = i * 0.1
        w2 = j * 0.1
        w3 = 1 -w1-w2
        if w1<-0.01 or w2<-0.01 or w3<-0.01:
            break
        train_kernel = w1 * sig_mat + w2 * rbf_mat + w3 * poly_mat
        clf = SVC(kernel='precomputed', probability=True)
        acc = cross_val_score(clf, train_kernel, y, cv=5, scoring="accuracy")
        acc_avg = (acc[0] + acc[1] + acc[2])/3
        if acc_avg > best_score:
            best_w1 = w1
            best_w2 = w2
            best_w3 = w3
            best_score = acc_avg
        
print("w1:"+str(best_w1), "w2:"+str(best_w2), "w3:"+str(best_w3))
print("score:"+str(best_score))

w1:0.0 w2:0.0 w3:1.0
score:0.9206896551724139


In [6]:
train_kernel = best_w1 * sig_mat + best_w2 * rbf_mat + best_w3 * poly_mat
y_pred = cross_val_predict(clf, train_kernel, y, cv=5)
auc = cross_val_score(clf, train_kernel, y, cv=5, scoring="roc_auc")
print(confusion_matrix(y, y_pred,labels=[1,0]))
print(classification_report(y, y_pred))
auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
print("auc: "+str(auc_avg)+"\n")

[[41  5]
 [ 8 92]]
              precision    recall  f1-score   support

           0       0.95      0.92      0.93       100
           1       0.84      0.89      0.86        46

    accuracy                           0.91       146
   macro avg       0.89      0.91      0.90       146
weighted avg       0.91      0.91      0.91       146

auc: 0.9758888888888888



## 3. ElasticNet with Classifiers

### 3.1 ElasticNet

In [7]:
model = ElasticNetCV(random_state=0)
#,alphas=[1,0.1,0.01,0.005,0.001,0.0001]
model.fit(X, y)

print("alpha: " + str(model.alpha_))
#print(els.coef_)

selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected = selector.transform(X)
print("X_selected: " + str(X_selected.shape))

alpha: 34.48014261587542
X_selected: (146, 18)


### 3.2 Classifiers

In [8]:
for i in range(5):
    # SVM
    if i == 0:
        clf = SVC(C=1, random_state=42, probability=True)
        print("SVM")
        
        
    # KNN
    elif i == 1:
        clf = neighbors.KNeighborsClassifier(n_neighbors=2)
        print("KNN")
        
    # Gaussian Naive Bayes
    elif i == 2:
        clf = GaussianNB()
        print("Gaussian Naive Bayes")
        
    # Decision Tree
    elif i == 3:
        clf = tree.DecisionTreeClassifier()
        print("Decision Tree")
        
    # Random Forest
    else:
        clf = RandomForestClassifier()
        print("Random Forest")
    
    # Model Evaluation
    y_pred = cross_val_predict(clf, X_selected, y, cv=5)
    auc = cross_val_score(clf, X_selected, y, cv=5, scoring="roc_auc")
    print(confusion_matrix(y, y_pred,labels=[1,0]))
    print(classification_report(y, y_pred))
    auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
    print("auc: "+str(auc_avg)+"\n")

SVM
[[  0  46]
 [  0 100]]
              precision    recall  f1-score   support

           0       0.68      1.00      0.81       100
           1       0.00      0.00      0.00        46

    accuracy                           0.68       146
   macro avg       0.34      0.50      0.41       146
weighted avg       0.47      0.68      0.56       146

auc: 0.9296666666666666

KNN
[[ 8 38]
 [ 6 94]]
              precision    recall  f1-score   support

           0       0.71      0.94      0.81       100
           1       0.57      0.17      0.27        46

    accuracy                           0.70       146
   macro avg       0.64      0.56      0.54       146
weighted avg       0.67      0.70      0.64       146

auc: 0.6086666666666667

Gaussian Naive Bayes
[[34 12]
 [ 7 93]]
              precision    recall  f1-score   support

           0       0.89      0.93      0.91       100
           1       0.83      0.74      0.78        46

    accuracy                           0.8

/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[35 11]
 [ 4 96]]
              precision    recall  f1-score   support

           0       0.90      0.96      0.93       100
           1       0.90      0.76      0.82        46

    accuracy                           0.90       146
   macro avg       0.90      0.86      0.88       146
weighted avg       0.90      0.90      0.89       146

auc: 0.986



### 3.3 Multi-core SVM

In [9]:
sig_mat = sigmoid_kernel(X_selected)
rbf_mat = rbf_kernel(X_selected)
poly_mat = polynomial_kernel(X_selected)

best_score = 0
best_w1 = 0
best_w2 = 0
best_w3 = 0

for i in range(11):
    for j in range(11):
        w1 = i * 0.1
        w2 = j * 0.1
        w3 = 1 -w1-w2
        if w1<-0.01 or w2<-0.01 or w3<-0.01:
            break
        train_kernel = w1 * sig_mat + w2 * rbf_mat + w3 * poly_mat
        clf = SVC(kernel='precomputed', probability=True)
        acc = cross_val_score(clf, train_kernel, y, cv=5, scoring="accuracy")
        acc_avg = (acc[0] + acc[1] + acc[2])/3
        if acc_avg > best_score:
            best_w1 = w1
            best_w2 = w2
            best_w3 = w3
            best_score = acc_avg
        
print("w1:"+str(best_w1), "w2:"+str(best_w2), "w3:"+str(best_w3))
print("score:"+str(best_score))

w1:0.0 w2:0.0 w3:1.0
score:0.9091954022988507


In [10]:
train_kernel = best_w1 * sig_mat + best_w2 * rbf_mat + best_w3 * poly_mat
y_pred = cross_val_predict(clf, train_kernel, y, cv=5)
auc = cross_val_score(clf, train_kernel, y, cv=5, scoring="roc_auc")
print(confusion_matrix(y, y_pred,labels=[1,0]))
print(classification_report(y, y_pred))
auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
print("auc: "+str(auc_avg)+"\n")

[[41  5]
 [ 8 92]]
              precision    recall  f1-score   support

           0       0.95      0.92      0.93       100
           1       0.84      0.89      0.86        46

    accuracy                           0.91       146
   macro avg       0.89      0.91      0.90       146
weighted avg       0.91      0.91      0.91       146

auc: 0.9672222222222222

